# Training

In [ ]:
!pip install unsloth

In [ ]:
import unsloth

import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
# import torch
# import transformers
# from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed, BitsAndBytesConfig
from datasets import load_dataset, Dataset, DatasetDict
import wandb
#from peft import LoraConfig
#from trl import SFTTrainer, SFTConfig
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
# Constants
BASE_MODEL = "unsloth/phi-4-unsloth-bnb-4bit"

PROJECT_NAME = "pricer"
HF_USER = "javiomotero" # your HF name here!

DATASET_NAME = f"{HF_USER}/lite-data"

dataset = load_dataset(DATASET_NAME)
train = dataset['train']
test = dataset['test']

# Split your dataset into train and eval
split_dataset = train.train_test_split(test_size=0.1, seed=42)

train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]



RUN_NAME =  f"{datetime.now():%Y-%m-%d_%H.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"



LOGGING_STEPS = 50
SAVE_STEPS = 500
LOG_TO_WANDB = True

# Log in to HuggingFace

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

# Log in to Weights & Biases
wandb_api_key = userdata.get('WANDB_API_KEY')
os.environ["WANDB_API_KEY"] = wandb_api_key
wandb.login()

# Configure Weights & Biases to record against our project
os.environ["WANDB_PROJECT"] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ["WANDB_WATCH"] = "gradients"

if LOG_TO_WANDB:
  run = wandb.init(project=PROJECT_NAME, name=RUN_NAME)

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
import torch


# Your hyperparameters
LORA_R = 8
LORA_ALPHA = 2 * LORA_R
LORA_DROPOUT = 0.2
TARGET_MODULES = ["q_proj", "v_proj", "k_proj", "o_proj"]  # keep small for T4


EPOCHS = 1
BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 1
LEARNING_RATE = 1e-4
LR_SCHEDULER_TYPE = "cosine"
WARMUP_RATIO = 0.03
OPTIMIZER = "paged_adamw_32bit"  # consider adamw_8bit if you hit NaNs or OOM
MAX_SEQUENCE_LENGTH = 182

# 1) Load model via Unsloth in 4-bit
dtype = "bfloat16" if is_bfloat16_supported() else "float16"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL,
    max_seq_length = MAX_SEQUENCE_LENGTH,
    load_in_4bit = True,
    dtype = dtype,
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 2) Apply LoRA using Unsloth helper (uses gradient checkpointing under the hood if set)
peft_config = LoraConfig(
    r = LORA_R,
    lora_alpha = LORA_ALPHA,
    lora_dropout = LORA_DROPOUT,
    bias = "none",
    task_type = "CAUSAL_LM",
    target_modules = TARGET_MODULES,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = peft_config.r,
    lora_alpha = peft_config.lora_alpha,
    lora_dropout = peft_config.lora_dropout,
    target_modules = peft_config.target_modules,
    bias = peft_config.bias,
    use_gradient_checkpointing = "unsloth",
)

# 3) Your SFTConfig (same API, Unsloth integrates with TRL’s SFTTrainer)
train_parameters = SFTConfig(
    output_dir = PROJECT_RUN_NAME,
    num_train_epochs = EPOCHS,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = 1,
    eval_strategy = "steps",
    eval_steps = SAVE_STEPS,
    gradient_accumulation_steps = GRADIENT_ACCUMULATION_STEPS,
    optim = OPTIMIZER,
    save_steps = SAVE_STEPS,
    save_total_limit = 10,
    logging_steps = LOGGING_STEPS,
    learning_rate = LEARNING_RATE,
    weight_decay = 0.001,
    fp16 = (dtype == "float16"),
    bf16 = (dtype == "bfloat16"),
    max_grad_norm = 0.3,
    max_steps = -1,
    warmup_ratio = WARMUP_RATIO,
    group_by_length = True,
    lr_scheduler_type = LR_SCHEDULER_TYPE,
    report_to = "wandb" if LOG_TO_WANDB else None,
    run_name = RUN_NAME,
    max_seq_length = MAX_SEQUENCE_LENGTH,
    dataset_text_field = "text",
    save_strategy = "steps",
    hub_strategy = "every_save",
    push_to_hub = True,
    hub_model_id = HUB_MODEL_NAME,
    hub_private_repo = True,
)



In [ ]:

#Checkpointing from wandb (run) - I guess we can also do from HF
#checkpoint_url = "javier-otero-marquez-personal-education/pricer/model-2025-10-25_15.39.13:v3" #This was for first retrain
checkpoint_url = "javier-otero-marquez-personal-education/pricer/model-2025-10-26_09.54.35:v1"

artifact = run.use_artifact(checkpoint_url, type='model')
artifact_dir = artifact.download()

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    args = train_parameters,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    packing = False,  # safer for stability; can turn on after it fits
    completion_only_loss=True
)
trainer.train(resume_from_checkpoint=artifact_dir)


# Inference

In [ ]:
# imports

import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import torch.nn.functional as F
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, set_seed
from datasets import load_dataset, Dataset, DatasetDict
from datetime import datetime
from peft import PeftModel
import matplotlib.pyplot as plt
# Constants

BASE_MODEL = "unsloth/phi-4-unsloth-bnb-4bit"
PROJECT_NAME = "pricer"
HF_USER = "javiomotero" # your HF name here! Or use mine if you just want to reproduce my results.

# The run itselfjaviomotero/pricer-
RUN_NAME = "2025-10-26_09.54.35"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
REVISION = "53c8d992140e5b184e9388418d711d3e38f7bd9d" # or REVISION = None
FINETUNED_MODEL = f"{HF_USER}/{PROJECT_RUN_NAME}"

# Uncomment this line if you wish to use my model
# FINETUNED_MODEL = f"ed-donner/{PROJECT_RUN_NAME}"

# Data

DATASET_NAME = f"{HF_USER}/lite-data"
# Or just use the one I've uploaded
# DATASET_NAME = "ed-donner/pricer-data"

# Hyperparameters for QLoRA

QUANT_4_BIT = True

%matplotlib inline

# Used for writing to output in color

GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
RESET = "\033[0m"
COLOR_MAP = {"red":RED, "orange": YELLOW, "green": GREEN}
# Log in to HuggingFace

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)
dataset = load_dataset(DATASET_NAME)
train = dataset['train']
test = dataset['test']
# pick the right quantization (thank you Robert M. for spotting the bug with the 8 bit version!)

if QUANT_4_BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )
# Load the Tokenizer and the Model

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="auto",
)
base_model.generation_config.pad_token_id = tokenizer.pad_token_id

# Load the fine-tuned model with PEFT
if REVISION:
  fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL, revision=REVISION)
else:
  fine_tuned_model = PeftModel.from_pretrained(base_model, FINETUNED_MODEL)


print(f"Memory footprint: {fine_tuned_model.get_memory_footprint() / 1e6:.1f} MB")

In [ ]:
def extract_price(s):
    if "Price is $" in s:
      contents = s.split("Price is $")[1]
      contents = contents.replace(',','')
      match = re.search(r"[-+]?\d*\.\d+|\d+", contents)
      return float(match.group()) if match else 0
    return 0
top_K = 3

def improved_model_predict(prompt, device="cuda"):
    set_seed(42)
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    attention_mask = torch.ones(inputs.shape, device=device)

    with torch.no_grad():
        outputs = fine_tuned_model(inputs, attention_mask=attention_mask)
        next_token_logits = outputs.logits[:, -1, :].to('cpu')

    next_token_probs = F.softmax(next_token_logits, dim=-1)
    top_prob, top_token_id = next_token_probs.topk(top_K)
    prices, weights = [], []
    for i in range(top_K):
      predicted_token = tokenizer.decode(top_token_id[0][i])
      probability = top_prob[0][i]
      try:
        result = float(predicted_token)
      except ValueError as e:
        result = 0.0
      if result > 0:
        prices.append(result)
        weights.append(probability)
    if not prices:
      return 0.0, 0.0
    total = sum(weights)
    weighted_prices = [price * weight / total for price, weight in zip(prices, weights)]
    return sum(weighted_prices).item()

class Tester:

    def __init__(self, predictor, data, title=None, size=250):
        self.predictor = predictor
        self.data = data
        self.title = title or predictor.__name__.replace("_", " ").title()
        self.size = size
        self.guesses = []
        self.truths = []
        self.errors = []
        self.sles = []
        self.colors = []

    def color_for(self, error, truth):
        if error<40 or error/truth < 0.2:
            return "green"
        elif error<80 or error/truth < 0.4:
            return "orange"
        else:
            return "red"

    def run_datapoint(self, i):
        datapoint = self.data[i]
        guess = self.predictor(datapoint["text"])
        truth = datapoint["price"]
        error = abs(guess - truth)
        log_error = math.log(truth+1) - math.log(guess+1)
        sle = log_error ** 2
        color = self.color_for(error, truth)
        title = datapoint["text"].split("\n\n")[1][:20] + "..."
        self.guesses.append(guess)
        self.truths.append(truth)
        self.errors.append(error)
        self.sles.append(sle)
        self.colors.append(color)
        print(f"{COLOR_MAP[color]}{i+1}: Guess: ${guess:,.2f} Truth: ${truth:,.2f} Error: ${error:,.2f} SLE: {sle:,.2f} Item: {title}{RESET}")

    def chart(self, title):
        max_error = max(self.errors)
        plt.figure(figsize=(12, 8))
        max_val = max(max(self.truths), max(self.guesses))
        plt.plot([0, max_val], [0, max_val], color='deepskyblue', lw=2, alpha=0.6)
        plt.scatter(self.truths, self.guesses, s=3, c=self.colors)
        plt.xlabel('Ground Truth')
        plt.ylabel('Model Estimate')
        plt.xlim(0, max_val)
        plt.ylim(0, max_val)
        plt.title(title)
        plt.show()

    def report(self):
        average_error = sum(self.errors) / self.size
        rmsle = math.sqrt(sum(self.sles) / self.size)
        hits = sum(1 for color in self.colors if color=="green")
        title = f"{self.title} Error=${average_error:,.2f} RMSLE={rmsle:,.2f} Hits={hits/self.size*100:.1f}%"
        self.chart(title)

    def run(self):
        self.error = 0
        for i in range(self.size):
            self.run_datapoint(i)
        self.report()

    @classmethod
    def test(cls, function, data):
        cls(function, data).run()
#Step 6000
Tester.test(improved_model_predict, test)